In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
import requests
import datetime
import time
import pandas as pd

StatementMeta(, c944ef61-1ca7-45f4-8daa-687f4ee5293c, 3, Finished, Available, Finished)

In [2]:
# 1. Configuración de Ciudades
ciudades_latam = [
    {"ciudad": "Medellin", "lat": 6.2442, "lon": -75.5812},
    {"ciudad": "Cali", "lat": 3.4516, "lon": -76.5320},
    {"ciudad": "Barranquilla", "lat": 10.9685, "lon": -74.7813},
    {"ciudad": "Bogota", "lat": 4.7110, "lon": -74.0721},
    {"ciudad": "Sao Paulo", "lat": -23.5505, "lon": -46.6333},
    {"ciudad": "Mexico City", "lat": 19.4326, "lon": -99.1332},
    {"ciudad": "Buenos Aires", "lat": -34.6037, "lon": -58.3816},
    {"ciudad": "Santiago", "lat": -33.4489, "lon": -70.6693},
    {"ciudad": "Lima", "lat": -12.0464, "lon": -77.0428},
    {"ciudad": "Guayaquil", "lat": -2.1894, "lon": -79.8891}
]

StatementMeta(, c944ef61-1ca7-45f4-8daa-687f4ee5293c, 4, Finished, Available, Finished)

In [3]:
API_KEY = "82667709d0d19d53a509356e055cb470"
LAKEHOUSE_NAME = "Lk_Bronce"
TABLE_NAME = "clima_historico"
PATH_PARQUET = f"Files/{TABLE_NAME}" # Ruta en la sección 'Files' del Lakehouse

StatementMeta(, c944ef61-1ca7-45f4-8daa-687f4ee5293c, 5, Finished, Available, Finished)

In [4]:
# 2. Generar rango del ÚLTIMO MES (30 días atrás hasta ayer)
fecha_fin = datetime.datetime.now() - datetime.timedelta(days=10)
fecha_inicio = fecha_fin - datetime.timedelta(days=30)

rango_fechas = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='D')
timestamps_mes = [int(d.replace(hour=12).timestamp()) for d in rango_fechas]


print(f"DEBUG: Fecha Inicio: {fecha_inicio}")
print(f"DEBUG: Fecha Fin: {fecha_fin}")
print(f"DEBUG: Cantidad de días detectados: {len(rango_fechas)}")

if len(rango_fechas) == 0:
    print("❌ ERROR: El rango de fechas está vacío. Revisa la configuración de datetime.")

print(f"Procesando desde {fecha_inicio.date()} hasta {fecha_fin.date()} ({len(timestamps_mes)} días)")

StatementMeta(, c944ef61-1ca7-45f4-8daa-687f4ee5293c, 6, Finished, Available, Finished)

DEBUG: Fecha Inicio: 2025-12-07 16:51:40.901145
DEBUG: Fecha Fin: 2026-01-06 16:51:40.901145
DEBUG: Cantidad de días detectados: 31
Procesando desde 2025-12-07 hasta 2026-01-06 (31 días)


In [5]:
# 2. PRUEBA DE CONEXIÓN (Solo 1 ciudad, 1 día)
lat, lon = 6.2442, -75.5812 # Medellin
ts_test = int(fecha_fin.timestamp())
url_test = f"https://api.openweathermap.org/data/3.0/history/timemachine?lat={lat}&lon={lon}&dt={ts_test}&appid={API_KEY}"

print(f"\nDEBUG: Probando conexión a URL: {url_test}")

try:
    r = requests.get(url_test, timeout=10)
    print(f"DEBUG: Status Code de la API: {r.status_code}")
    print(f"DEBUG: Respuesta cruda: {r.text}")
    
    if r.status_code == 403:
        print("\n💡 EXPLICACIÓN: Tienes error 403. Esto significa que tu API KEY funciona, pero no tienes activado el plan 'One Call 3.0' en la web de OpenWeather.")
    elif r.status_code == 401:
        print("\n💡 EXPLICACIÓN: Tienes error 401. Tu API KEY es incorrecta o no ha sido activada aún.")
        
except Exception as e:
    print(f"❌ ERROR DE RED: {e}")

StatementMeta(, c944ef61-1ca7-45f4-8daa-687f4ee5293c, 7, Finished, Available, Finished)


DEBUG: Probando conexión a URL: https://api.openweathermap.org/data/3.0/history/timemachine?lat=6.2442&lon=-75.5812&dt=1767718300&appid=82667709d0d19d53a509356e055cb470
DEBUG: Status Code de la API: 404
DEBUG: Respuesta cruda: {"cod":"404","message":"Internal error"}


In [6]:
# 3. Función de consulta optimizada para diagnóstico
def obtener_datos(lat, lon, dt):
    url = f"https://api.openweathermap.org/data/3.0/history/timemachine?lat={lat}&lon={lon}&dt={dt}&appid={API_KEY}&units=metric"
    try:
        r = requests.get(url, timeout=10)
        
        if r.status_code == 200:
            return r.json()
            print(r)
        else:
            # Esto imprimirá el error real en la consola del Notebook
            print(f"❌ Error API ({r.status_code}): {r.text} | URL: {url}")
            return None
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Error de conexión: {e}")
        return None


StatementMeta(, c944ef61-1ca7-45f4-8daa-687f4ee5293c, 8, Finished, Available, Finished)

In [7]:
# 4. Bucle de recolección
registros = []
for c in ciudades_latam:
    print(f"Consultando {c['ciudad']}...")
    for ts in timestamps_mes:
        res = obtener_datos(c['lat'], c['lon'], ts)
        if res and 'data' in res:
            d = res['data'][0]
            registros.append((
                c['ciudad'], c['lat'], c['lon'],
                float(d['temp']), int(d['humidity']),
                d['weather'][0]['description'],
                datetime.datetime.fromtimestamp(ts)
            ))
        time.sleep(0.1) # Breve pausa para estabilidad

StatementMeta(, c944ef61-1ca7-45f4-8daa-687f4ee5293c, 9, Finished, Available, Finished)

Consultando Medellin...
❌ Error API (404): {"cod":"404","message":"Internal error"} | URL: https://api.openweathermap.org/data/3.0/history/timemachine?lat=6.2442&lon=-75.5812&dt=1765111900&appid=82667709d0d19d53a509356e055cb470&units=metric
❌ Error API (404): {"cod":"404","message":"Internal error"} | URL: https://api.openweathermap.org/data/3.0/history/timemachine?lat=6.2442&lon=-75.5812&dt=1765198300&appid=82667709d0d19d53a509356e055cb470&units=metric
❌ Error API (404): {"cod":"404","message":"Internal error"} | URL: https://api.openweathermap.org/data/3.0/history/timemachine?lat=6.2442&lon=-75.5812&dt=1765284700&appid=82667709d0d19d53a509356e055cb470&units=metric
❌ Error API (404): {"cod":"404","message":"Internal error"} | URL: https://api.openweathermap.org/data/3.0/history/timemachine?lat=6.2442&lon=-75.5812&dt=1765371100&appid=82667709d0d19d53a509356e055cb470&units=metric
❌ Error API (404): {"cod":"404","message":"Internal error"} | URL: https://api.openweathermap.org/data/3.0/h

In [8]:
# Verificamos si la lista tiene datos antes de intentar crear el DataFrame
if len(registros) > 0:
    df_mes = spark.createDataFrame(registros, schema)
    df_mes.write.format("delta").mode("append").saveAsTable(TABLE_NAME)
    print(f"Se insertaron {len(registros)} filas correctamente.")
else:
    print("La lista de registros está vacía. Revisa los errores de la API arriba.")

StatementMeta(, c944ef61-1ca7-45f4-8daa-687f4ee5293c, 10, Finished, Available, Finished)

La lista de registros está vacía. Revisa los errores de la API arriba.


In [9]:
# 5. Crear DataFrame y guardar en el Lakehouse
schema = StructType([
    StructField("Ciudad", StringType(), True),
    StructField("Latitud", DoubleType(), True),
    StructField("Longitud", DoubleType(), True),
    StructField("Temperatura", DoubleType(), True),
    StructField("Humedad", IntegerType(), True),
    StructField("Descripcion", StringType(), True),
    StructField("Fecha", TimestampType(), True)
])

df_mes = spark.createDataFrame(registros, schema)

# Solución al AnalysisException: 
# Usamos 'overwrite' si es la primera carga o 'append' si ya tienes datos.
# Fabric prefiere que escribas a la tabla directamente si está adjunto.
try:
    df_mes.write.format("delta").mode("append").saveAsTable(TABLE_NAME)
    print("Carga del último mes completada exitosamente en Lk_Bronce.")
except Exception as e:
    print(f"Error al guardar: {e}")
    # Alternativa manual por si falla el metastore:
    # df_mes.write.format("delta").mode("append").save(f"Tables/{TABLE_NAME}")

StatementMeta(, c944ef61-1ca7-45f4-8daa-687f4ee5293c, 11, Submitted, Running, Running)

In [10]:
import requests
import datetime

# 1. Ajuste de seguridad: Pedir datos de hace 48 horas para asegurar disponibilidad
fecha_segura = datetime.datetime.now() - datetime.timedelta(days=2)
ts_seguro = int(fecha_segura.timestamp())

API_KEY = "82667709d0d19d53a509356e055cb470"
lat, lon = 6.2442, -75.5812

url_test = f"https://api.openweathermap.org/data/3.0/history/timemachine?lat={lat}&lon={lon}&dt={ts_seguro}&appid={API_KEY}&units=metric"

print(f"Probando con fecha: {fecha_segura.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Timestamp: {ts_seguro}")

response = requests.get(url_test)

print(f"Status Code: {response.status_code}")
print(f"Respuesta: {response.text}")

if response.status_code == 404:
    print("\n⚠️ SIGUE EL ERROR 404:")
    print("Esto confirma que, o la fecha es muy reciente, o tu API KEY no tiene activo 'One Call 3.0'.")
    print("Prueba entrando aquí: https://home.openweathermap.org/subscriptions y verifica que diga 'One Call API 3.0' ACTIVE.")

StatementMeta(, 197c27fb-7ba0-4a44-98fd-da83c3c23bc5, 12, Finished, Available, Finished)

Probando con fecha: 2026-01-11 20:16:13
Timestamp: 1768162573
Status Code: 404
Respuesta: {"cod":"404","message":"Internal error"}

⚠️ SIGUE EL ERROR 404:
Esto confirma que, o la fecha es muy reciente, o tu API KEY no tiene activo 'One Call 3.0'.
Prueba entrando aquí: https://home.openweathermap.org/subscriptions y verifica que diga 'One Call API 3.0' ACTIVE.


In [11]:
import requests
import datetime
import time
from pyspark.sql.types import *

# 1. Configuración
API_KEY = "82667709d0d19d53a509356e055cb470"
TABLE_NAME = "clima_historico"

# Forzamos una fecha de hace 3 días para asegurar que el historial existe
fecha_prueba = datetime.datetime.now() - datetime.timedelta(days=3)
ts_prueba = int(fecha_prueba.replace(hour=12, minute=0).timestamp())

# Ciudad de prueba: Medellin
lat, lon = 6.2442, -75.5812

url = f"https://api.openweathermap.org/data/3.0/history/timemachine?lat={lat}&lon={lon}&dt={ts_prueba}&appid={API_KEY}&units=metric"

print(f"Consultando fecha: {fecha_prueba.date()} | Timestamp: {ts_prueba}")

# 2. Ejecución de la petición
registros = []
response = requests.get(url)

if response.status_code == 200:
    data = response.json()['data'][0]
    registros.append((
        "Medellin", lat, lon, float(data['temp']), 
        int(data['humidity']), data['weather'][0]['description'], 
        datetime.datetime.fromtimestamp(ts_prueba)
    ))
    
    # 3. Guardar en Lakehouse Lk_Bronce si hay datos
    schema = StructType([
        StructField("Ciudad", StringType(), True),
        StructField("Latitud", DoubleType(), True),
        StructField("Longitud", DoubleType(), True),
        StructField("Temperatura", DoubleType(), True),
        StructField("Humedad", IntegerType(), True),
        StructField("Descripcion", StringType(), True),
        StructField("Fecha", TimestampType(), True)
    ])
    
    df = spark.createDataFrame(registros, schema)
    # En Fabric, usamos la ruta Tables/ para asegurar que se registre en el Lakehouse
    df.write.format("delta").mode("append").saveAsTable(TABLE_NAME)
    
    print("✅ ¡Éxito! Datos guardados en la tabla 'clima_historico' de Lk_Bronce.")
else:
    print(f"❌ Error persistente: {response.status_code}")
    print(f"Mensaje: {response.text}")
    print("\nPASO NECESARIO: Si el error es 404 o 401, debes entrar a https://home.openweathermap.org/subscriptions y activar el plan 'One Call API 3.0'. Sin eso, el historial no funcionará.")

StatementMeta(, 197c27fb-7ba0-4a44-98fd-da83c3c23bc5, 13, Finished, Available, Finished)

Consultando fecha: 2026-01-10 | Timestamp: 1768046450
❌ Error persistente: 404
Mensaje: {"cod":"404","message":"Internal error"}

PASO NECESARIO: Si el error es 404 o 401, debes entrar a https://home.openweathermap.org/subscriptions y activar el plan 'One Call API 3.0'. Sin eso, el historial no funcionará.
